# 02_Preprocessamento.ipynb
---------------------------------
1. Imports e Configurações
---------------------------------

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")


In [ ]:
# Célula 2 — CORRIGIR CAMINHOS

RAW = r"C:\Users\levie\asasas\projeto_ml\data\raw\delivery_time.csv"
CLEAN = r"C:\Users\levie\asasas\projeto_ml\data\processed\delivery_clean.csv"
SCALED = r"C:\Users\levie\asasas\projeto_ml\data\processed\delivery_processed.csv"


---------------------------------
2. Caminhos dos Arquivos + Carregar arquivos
---------------------------------

In [ ]:
RAW = r"C:\Users\levie\asasas\projeto_ml\data\raw\delivery_time.csv"
CLEAN = r"C:\Users\levie\asasas\projeto_ml\data\processed\delivery_clean.csv"
SCALED = r"C:\Users\levie\asasas\projeto_ml\data\processed\delivery_processed.csv"

# carregar raw (erro se faltar — arquivo raw é obrigatório)
if not os.path.exists(RAW):
    raise FileNotFoundError(f"Arquivo RAW não encontrado: {RAW}")
df_raw = pd.read_csv(RAW)

# carregar clean/processed se existirem; caso contrário criar cópias a partir do raw
if os.path.exists(CLEAN):
    df_clean = pd.read_csv(CLEAN)
else:
    print("WARNING: arquivo CLEAN não encontrado — usando cópia de raw como df_clean (execute células de preprocessamento).")
    df_clean = df_raw.copy()

if os.path.exists(SCALED):
    df_scaled = pd.read_csv(SCALED)
else:
    print("WARNING: arquivo SCALED não encontrado — usando df_clean como df_scaled (execute pré-processamento).")
    df_scaled = df_clean.copy()

print("Shapes — raw:", df_raw.shape, "clean:", df_clean.shape, "scaled:", df_scaled.shape)
display(df_raw.head())
display(df_clean.head())
display(df_scaled.head())



---------------------------------
4. Verificar Valores Negativos
---------------------------------

In [ ]:
cols_to_check = [
    "distance_km",
    "package_weight_kg",
    "driver_experience_years",
    "num_stops",
    "customer_rating",
    "fuel_cost",
    "delivery_time_hours"
]

for col in cols_to_check:
    neg_count = (df_clean[col] < 0).sum()
    if neg_count > 0:
        print(f"⚠️ {col} possui {neg_count} valores negativos!")
    else:
        print(f"✔️ {col} está sem valores negativos.")


---------------------------------
5. Gráfico de Comparação (Raw vs Clean vs Scaled)
---------------------------------

In [ ]:
col = "distance_km"  

plt.figure(figsize=(10,5))
sns.kdeplot(df_raw[col], label="Raw (etapa 1)", fill=True)
sns.kdeplot(df_clean[col], label="Clean (etapa 2)", fill=True)
sns.kdeplot(df_scaled[col], label="Scaled (etapa 3)", fill=True)

plt.title(f"Comparação de Distribuições — {col}")
plt.legend()
plt.show()


---------------------------------
6. Resolver Valores Negativos
---------------------------------

In [ ]:
# Criar diretório se não existir
os.makedirs(os.path.dirname(CLEAN), exist_ok=True)

for col in ["distance_km", "package_weight_kg"]:
    df_clean[col] = df_clean[col].abs()

df_clean.to_csv(CLEAN, index=False)
print("Valores negativos corrigidos e arquivo salvo.")


---------------------------------
7. Revalidar Após a Correção
---------------------------------

In [ ]:
import numpy as np

# usa cols_fix se definida; caso contrário tenta cols_to_check; senão usa todas as numéricas em df_clean
cols_fix = globals().get('cols_fix') or globals().get('cols_to_check')
if cols_fix is None:
    cols_fix = [c for c in df_clean.select_dtypes(include=[np.number]).columns.tolist()]

print("Revalidando colunas:", cols_fix)
for col in cols_fix:
    if col not in df_clean.columns:
        print(f"⚠️ Coluna '{col}' não encontrada em df_clean — pulando.")
        continue
    neg_remaining = (df_clean[col] < 0).sum()
    print(f"{col}: negativos restantes = {neg_remaining}")


In [ ]:
# Célula 7.5 — Criar arquivo SCALED com StandardScaler

from sklearn.preprocessing import StandardScaler
import joblib

# Definir colunas numéricas para escalar
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()

# Criar cópia para escalar
df_scaled = df_clean.copy()

# Criar e ajustar o scaler
scaler = StandardScaler()
df_scaled[numeric_cols] = scaler.fit_transform(df_clean[numeric_cols])

# Criar diretórios se não existirem
os.makedirs(os.path.dirname(SCALED), exist_ok=True)
os.makedirs(r"C:\Users\levie\asasas\projeto_ml\models", exist_ok=True)

# Salvar arquivos
df_scaled.to_csv(SCALED, index=False)
joblib.dump(scaler, r"C:\Users\levie\asasas\projeto_ml\models\scaler.pkl")

# Salvar metadata
meta = {"numeric_to_scale": numeric_cols}
joblib.dump(meta, r"C:\Users\levie\asasas\projeto_ml\models\preprocess_meta.pkl")

print("✔️ Arquivo SCALED salvo!")
print("✔️ Scaler salvo em models/scaler.pkl")
print("✔️ Metadata salvo em models/preprocess_meta.pkl")
print("Colunas escaladas:", numeric_cols)


---------------------------------
8. Gráfico de Comparação Após Correção
---------------------------------

In [ ]:
col = "distance_km" 

plt.figure(figsize=(10,5))
sns.kdeplot(df_raw[col], label="Raw (etapa 1)", fill=True)
sns.kdeplot(df_clean[col], label="Clean corrigido (etapa 2)", fill=True)
sns.kdeplot(df_scaled[col], label="Scaled (etapa 3)", fill=True)

plt.title(f"Distribuição — RAW vs CLEAN CORRIGIDO vs SCALED — {col}")
plt.legend()
plt.show()


---------------------------------
9. Mostrar Estatísticas das Três Versões
---------------------------------

In [ ]:
# CÉLULA 09 — Estatísticas principais
pd.DataFrame({
    "Raw": df_raw.describe().iloc[1],
    "Clean Corrigido": df_clean.describe().iloc[1],
    "Scaled": df_scaled.describe().iloc[1]
})


---------------------------------
10. Teste Scaler
---------------------------------

In [ ]:
# ===========================================
# 🔎 CÉLULA ÚNICA — DIAGNÓSTICO COMPLETO DO SCALER
# ===========================================

import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

RAW = r"C:\Users\levie\asasas\projeto_ml\data\raw\delivery_time.csv"
OUT_CLEAN = r"C:\Users\levie\asasas\projeto_ml\data\processed\delivery_clean.csv"
OUT_PROCESSED = r"C:\Users\levie\asasas\projeto_ml\data\processed\delivery_processed.csv"
SCALER_PATH = r"C:\Users\levie\asasas\projeto_ml\models\scaler.pkl"
META_PATH = r"C:\Users\levie\asasas\projeto_ml\models\preprocess_meta.pkl" 

# --- coluna que você quer diagnosticar
col = "distance_km"     # 👈 TROQUE AQUI

# --- carregar dados e objetos
df_raw = pd.read_csv(RAW)
df_clean = pd.read_csv(OUT_CLEAN)
df_processed = pd.read_csv(OUT_PROCESSED)
scaler = joblib.load(SCALER_PATH)
meta = joblib.load(META_PATH)

numeric_to_scale = meta.get("numeric_to_scale", [])

print("\n==============================")
print("📌 INFORMAÇÕES INICIAIS")
print("==============================")
print("Coluna analisada:", col)
print("Colunas escaladas:", numeric_to_scale)
print("Existe no processed:", col in df_processed.columns)

if col not in df_raw.columns:
    print(f"\n❌ ERRO: coluna {col} não existe no RAW. Verifique o nome.")
    raise SystemExit()

if col not in df_clean.columns:
    print(f"\n❌ ERRO: coluna {col} não existe no CLEAN. Verifique o nome.")
    raise SystemExit()

# =======================================================
# TESTE 1 — Estatísticas antes/depois
# =======================================================
print("\n==============================")
print("📊 TESTE 1 — Estatísticas RAW vs CLEAN vs PROCESSED")
print("==============================")

orig = df_clean[col]
proc = df_processed[col] if col in df_processed.columns else None

print("RAW   → min / median / mean / max:", 
      df_raw[col].min(), df_raw[col].median(), df_raw[col].mean(), df_raw[col].max())
print("CLEAN → min / median / mean / max:", 
      orig.min(), orig.median(), orig.mean(), orig.max())

if proc is not None:
    print("PROC  → min / median / mean / max:",
          proc.min(), proc.median(), proc.mean(), proc.max())
else:
    print("PROC  → coluna não está no df_processed (possível one-hot).")

# =======================================================
# TESTE 2 — Plotar distribuições
# =======================================================
print("\n==============================")
print("📈 TESTE 2 — Gráficos de Distribuição")
print("==============================")

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
sns.histplot(orig.dropna(), kde=True)
plt.title(f"CLEAN — {col}")

plt.subplot(1,2,2)
if proc is not None:
    sns.histplot(proc.dropna(), kde=True)
    plt.title(f"PROCESSED (scaled) — {col}")
else:
    plt.text(0.3, 0.5, "Coluna não está no processed", fontsize=14)
plt.show()

# =======================================================
# TESTE 3 — Verificar se virou one-hot (dummies)
# =======================================================
print("\n==============================")
print("🧩 TESTE 3 — Verificando se a coluna virou dummies")
print("==============================")

related = [c for c in df_processed.columns if c.startswith(col + "_")]
if len(related) > 0:
    print("A coluna original virou as seguintes dummies:")
    print(related)
else:
    print("Nenhuma dummy encontrada — é uma coluna numérica normal.")

# =======================================================
# TESTE 4 — Inversão do scaling
# =======================================================
print("\n==============================")
print("🔄 TESTE 4 — Inversão do StandardScaler")
print("==============================")

if col in numeric_to_scale:
    idx = numeric_to_scale.index(col)
    scaled_vals = df_processed[col].to_numpy()
    inv = scaled_vals * scaler.scale_[idx] + scaler.mean_[idx]

    print("Valores invertidos (min/median/max):",
          np.nanmin(inv), np.nanmedian(inv), np.nanmax(inv))
else:
    print("A coluna NÃO está em numeric_to_scale → não passou pelo scaler.")

# =======================================================
# DIAGNÓSTICO FINAL
# =======================================================
print("\n==============================")
print("🧠 DIAGNÓSTICO FINAL")
print("==============================")

dummy_flag = len(related) > 0
scaled_flag = col in numeric_to_scale

if dummy_flag:
    print("✔ A coluna virou várias colunas one-hot ⚠️")
    print("Picos altos são NORMAIS em distribuições one-hot + scaler.")
elif not scaled_flag and col in df_processed:
    print("⚠ A coluna existe no processed mas NÃO está na lista numeric_to_scale.")
    print("Isso pode gerar comportamento estranho no gráfico.")
elif scaled_flag:
    print("✔ A coluna foi escalada corretamente.")
    print("Se existe pico, provavelmente é:")
    print("- distribuição muito concentrada (normal);")
    print("- valores discretos;")
    print("- ou outliers que foram limitados.")
else:
    print("❌ Algo está inconsistente: coluna não aparece no processed.")
